In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import jieba
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import roc_curve, auc, classification_report,accuracy_score,precision_score,f1_score,recall_score,cohen_kappa_score,multilabel_confusion_matrix,make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import metrics
import joblib
from scipy import sparse
from sklearn import ensemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.model_selection import cross_val_score,cross_validate

In [2]:
# load data 
dataframe = pd.read_csv(r'C:\Users\dawns\Desktop\Master thesis\data\Features_level.csv')
#data=dataframe[['CONTENT_NEW','','SCORE']].copy()
df_data=dataframe[['CONTENT_NEW','WORDCUT','SCORE','LEVEL','Tokens','level1_tokens','level2_tokens','level3_tokens','level4_tokens','level5_tokens','level6_tokens','level7_tokens','char_num','high_level','primary_level','Connective_ratio','connective_SCORE','pre','connective_numbers']].copy()



In [3]:
# Generating 'y' for classification as well as regression
y = df_data['SCORE']

In [4]:

# generating x 
pre = df_data['pre']
# Tokenization
def preprocessing_sentence(x):
    words = jieba.cut(str(x).strip())
    return ' '.join(words)

In [9]:
# load tf-idf vector
vector = joblib.load(r'E:\All codes\Master code\vector.pkl') 
vector.shape


(3668, 36035)

In [10]:
# 特征合并 feature concatenate 
feature_set1 = df_data[['connective_SCORE',]]
feature_set2 = vector
print(feature_set1.shape)
print(feature_set2.shape)
train_features = sparse.hstack((feature_set2, feature_set1))
print('=============')
print(train_features.shape)

(3668, 1)
(3668, 36035)
(3668, 36036)


In [15]:
# 划分训练集 ，测试集 train_test split
x_train, x_test, y_train, y_test = train_test_split(train_features, y, test_size=0.2)
print(x_train.shape)
print(y_train.shape)

(2934, 36036)
(2934,)


In [8]:
#Tf-IDF + LR 
#评估
# （准确率、精确度、召回率、F1值、Kappa系数用于评估分类任务）
## 二分类问题，一般设为两类，正类/反类，或0、1. 所以有tp，fn，fp，tn。 t，f代表是否正确，p，n代表是正类还是反类
## 准确率（accuracy）：所有分类正确的样本数，除以样本总数）A= （tp+tn）/（tp+tn+fp+fn）
## 精确度（precision）精确度也叫查准率，判为正的为p，包括正确tp，和错误fp，其中tp即预测正确的比例 P= tp/tp+fp
## 召回率（recall） ： 查全率，判断正确的且为正的是tp，判断错误且为负（该正却负的），想知道tp占实际正确的比例 r= tp/tp+fn
## f1 越大越好，f1=2*tp/(2*tp+fp+fn)
## kappa 衡量分类精度，其计算基于混淆矩阵，目前用的是二次加权kappa，对错误的惩罚更重，应用更广泛
# 逻辑回归LR
lr = LogisticRegression()
lr.fit(x_train, y_train)
train_score = lr.score(x_train, y_train)
y_pred = lr.predict(x_test)
print("Train Accuracy:", train_score)
print("================================================")
y_pred = lr.predict(x_test)
test_score = accuracy_score(y_test, y_pred)
print("Test Accuracy:", test_score)
print("================================================")
acc_for_each_class = metrics.precision_score(y_test,y_pred,average=None)
print("acc_for_each_class:\n",acc_for_each_class)
print("================================================")
qwkappa = cohen_kappa_score(y_test,y_pred,weights='quadratic')
print("qwkappa:\n",qwkappa)
print("================================================")
print('Weighted precision', precision_score(y_test, y_pred, average='weighted'))
print('Weighted recall', recall_score(y_test, y_pred, average='weighted'))
print('Weighted f1-score', f1_score(y_test, y_pred, average='weighted'))
print("================================================")
classification_rep = classification_report(y_test,y_pred)
print("classification report: \n",classification_rep)
print("================================================")
confusion_mat = metrics.confusion_matrix(y_test, y_pred)
print(confusion_mat)


Train Accuracy: 0.6820040899795501
Test Accuracy: 0.26294277929155313
acc_for_each_class:
 [0.         0.         0.         0.26666667 0.20588235 0.15584416
 0.30971129 0.29411765 0.17241379 0.2        0.34375    0.        ]
qwkappa:
 0.45158256339515324
Weighted precision 0.22986524794922333
Weighted recall 0.26294277929155313
Weighted f1-score 0.21206348550745635
classification report: 
               precision    recall  f1-score   support

          40       0.00      0.00      0.00         5
          45       0.00      0.00      0.00        10
          50       0.00      0.00      0.00        23
          55       0.27      0.08      0.12        52
          60       0.21      0.30      0.25        69
          65       0.16      0.13      0.14        89
          70       0.31      0.73      0.44       161
          75       0.29      0.10      0.14       105
          80       0.17      0.14      0.15        72
          85       0.20      0.09      0.13        77
          9

D:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to

In [9]:
# randomforest classfier
rf = ensemble.RandomForestClassifier()
params = {'n_estimators': [50], 'max_depth':[100], 'max_features':[10]}
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)
train_score = rf.score(x_train, y_train)
test_score = accuracy_score(y_test, y_pred)
print("Train Accuracy:", train_score)
print("================================================")
print("Test Accuracy:", test_score)
print("================================================")
acc_for_each_class = metrics.precision_score(y_test,y_pred,average=None)
print("acc_for_each_class:\n",acc_for_each_class)
print("================================================")
qwkappa = cohen_kappa_score(y_test,y_pred,weights='quadratic')
print("qwkappa:\n",qwkappa)
print("================================================")
print('Weighted precision', precision_score(y_test, y_pred, average='weighted'))
print('Weighted recall', recall_score(y_test, y_pred, average='weighted'))
print('Weighted f1-score', f1_score(y_test, y_pred, average='weighted'))
print("================================================")
classification_rep = classification_report(y_test,y_pred)
print("classification report: \n",classification_rep)
print("================================================")
confusion_mat = metrics.confusion_matrix(y_test, y_pred)
print(confusion_mat)


Train Accuracy: 0.9993183367416496
Test Accuracy: 0.273841961852861
acc_for_each_class:
 [0.         0.         0.         0.1        0.2295082  0.18333333
 0.28458498 0.35       0.26470588 0.33333333 0.33333333 1.        ]
qwkappa:
 0.37285459554639744
Weighted precision 0.2729060911735205
Weighted recall 0.273841961852861
Weighted f1-score 0.1980556620529144
classification report: 
               precision    recall  f1-score   support

          40       0.00      0.00      0.00         5
          45       0.00      0.00      0.00        10
          50       0.00      0.00      0.00        23
          55       0.10      0.02      0.03        52
          60       0.23      0.20      0.22        69
          65       0.18      0.12      0.15        89
          70       0.28      0.89      0.43       161
          75       0.35      0.07      0.11       105
          80       0.26      0.12      0.17        72
          85       0.33      0.06      0.11        77
          90     

D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are

In [16]:
# support vector machine classfier
model = OneVsRestClassifier(svm.SVC(kernel='rbf',gamma='scale'))#linear
clt = model.fit(x_train,y_train)
y_pred = clt.predict(x_test)
test_score = accuracy_score(y_test, y_pred)
train_score = clt.score(x_train, y_train)
print("Train Accuracy:", train_score)
print("================================================")
print("Test Accuracy:", test_score)
print("================================================")
acc_for_each_class = metrics.precision_score(y_test,y_pred,average=None)
print("acc_for_each_class:\n",acc_for_each_class)
print("================================================")
qwkappa = cohen_kappa_score(y_test,y_pred,weights='quadratic')
print("qwkappa:\n",qwkappa)
print("================================================")
print('Weighted precision', precision_score(y_test, y_pred, average='weighted'))
print('Weighted recall', recall_score(y_test, y_pred, average='weighted'))
print('Weighted f1-score', f1_score(y_test, y_pred, average='weighted'))
print("================================================")



Train Accuracy: 0.9911383776414451
Test Accuracy: 0.273841961852861
acc_for_each_class:
 [0.5        0.         0.         0.125      0.20930233 0.20879121
 0.36956522 0.27536232 0.18333333 0.16       0.27118644 0.6       ]
qwkappa:
 0.48749263271201637
Weighted precision 0.2524248741902846
Weighted recall 0.273841961852861
Weighted f1-score 0.25320430338931144


In [17]:
classification_rep = classification_report(y_test,y_pred)
print("classification report: \n",classification_rep)
print("================================================")

classification report: 
               precision    recall  f1-score   support

          40       0.50      0.14      0.22         7
          45       0.00      0.00      0.00         7
          50       0.00      0.00      0.00        26
          55       0.12      0.12      0.12        32
          60       0.21      0.23      0.22        80
          65       0.21      0.19      0.20       102
          70       0.37      0.58      0.45       177
          75       0.28      0.18      0.22       105
          80       0.18      0.15      0.17        72
          85       0.16      0.14      0.15        59
          90       0.27      0.30      0.29        53
          95       0.60      0.21      0.32        14

    accuracy                           0.27       734
   macro avg       0.24      0.19      0.20       734
weighted avg       0.25      0.27      0.25       734



In [18]:
confusion_mat = metrics.confusion_matrix(y_test, y_pred)
print(confusion_mat)

[[  1   0   1   0   3   1   1   0   0   0   0   0]
 [  0   0   0   2   3   1   0   0   1   0   0   0]
 [  0   0   0   6   6   6   6   1   1   0   0   0]
 [  1   1   0   4   9   7   3   0   3   4   0   0]
 [  0   1   1   7  18  13  26   5   4   3   1   1]
 [  0   0   0   4  17  19  38   8   8   5   3   0]
 [  0   0   0   6  13  20 102  12   8  10   6   0]
 [  0   0   0   1   6   9  54  19   4   7   5   0]
 [  0   0   0   1   7   9  21  11  11   4   8   0]
 [  0   0   0   0   4   3  14   6   7   8  17   0]
 [  0   0   0   1   0   3  10   6   9   7  16   1]
 [  0   0   0   0   0   0   1   1   4   2   3   3]]


In [11]:
# 交叉验证 cross validation   logistic regression
scoring = {'accuracy': 'accuracy','qwkappa': make_scorer(cohen_kappa_score,weights='quadratic')}
scores = cross_validate(lr, x_train, y_train, cv=10, scoring=scoring)#10-fold cv
accuracy = scores['test_accuracy']
qwkappa = scores['test_qwkappa']
print(scores)
print('Mean accuracy:',accuracy.mean())
print('Mean qwkappa:',qwkappa.mean())

D:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result

{'fit_time': array([6.86287689, 7.10459065, 6.82955647, 7.38531947, 6.84365487,
       7.10789752, 7.02449775, 6.76126266, 7.27092361, 6.66103196]), 'score_time': array([0.00302744, 0.00498772, 0.        , 0.00396633, 0.00299048,
       0.        , 0.00299263, 0.00398588, 0.0101788 , 0.        ]), 'test_accuracy': array([0.27210884, 0.29251701, 0.26530612, 0.24829932, 0.27303754,
       0.27986348, 0.23890785, 0.2662116 , 0.27303754, 0.21843003]), 'test_qwkappa': array([0.39372632, 0.49341478, 0.47116567, 0.42550341, 0.5454941 ,
       0.56497458, 0.48589714, 0.53145647, 0.47798676, 0.45830286])}
Mean accuracy: 0.2627719347124516
Mean qwkappa: 0.4847922082867022


D:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
# random forest classification
scoring = {'accuracy': 'accuracy','qwkappa': make_scorer(cohen_kappa_score,weights='quadratic')}
scores = cross_validate(rf, x_train, y_train, cv=10, scoring=scoring)#10-fold cv
accuracy = scores['test_accuracy']
qwkappa = scores['test_qwkappa']
print(scores)
print('Mean accuracy:',accuracy.mean())
print('Mean qwkappa:',qwkappa.mean())

{'fit_time': array([6.91099072, 6.33911276, 6.81523252, 6.51449704, 6.29195237,
       6.77543211, 5.82020426, 6.50162125, 5.91023684, 5.7287519 ]), 'score_time': array([0.03786969, 0.03590417, 0.03132081, 0.0304544 , 0.03490567,
       0.03787756, 0.03042865, 0.03040695, 0.03048754, 0.0302875 ]), 'test_accuracy': array([0.24829932, 0.28911565, 0.30952381, 0.30272109, 0.29010239,
       0.24914676, 0.25255973, 0.26279863, 0.27986348, 0.2662116 ]), 'test_qwkappa': array([0.26555711, 0.35293084, 0.45696653, 0.44826854, 0.51302666,
       0.37418277, 0.45082013, 0.46971703, 0.4176456 , 0.40021602])}
Mean accuracy: 0.27503424578022334
Mean qwkappa: 0.4149331212217507


In [13]:
# support machine classification
scoring = {'accuracy': 'accuracy','qwkappa': make_scorer(cohen_kappa_score,weights='quadratic')}
scores = cross_validate(clt, x_train, y_train, cv=10, scoring=scoring)#10-fold cv
accuracy = scores['test_accuracy']
qwkappa = scores['test_qwkappa']
print(scores)
print('Mean accuracy:',accuracy.mean())
print('Mean qwkappa:',qwkappa.mean())

{'fit_time': array([42.90427732, 43.12479758, 42.12815499, 40.78293347, 41.13643122,
       39.88809729, 42.17229915, 42.89752722, 39.38815403, 39.54461765]), 'score_time': array([3.89368725, 4.04334474, 3.95563388, 3.94590163, 3.91600609,
       3.87904549, 4.36841106, 3.988832  , 3.87046456, 3.84914446]), 'test_accuracy': array([0.26530612, 0.29931973, 0.25510204, 0.25170068, 0.25255973,
       0.28327645, 0.23549488, 0.2559727 , 0.2559727 , 0.23549488]), 'test_qwkappa': array([0.45720795, 0.63787422, 0.55837238, 0.46953558, 0.60214457,
       0.54416859, 0.50640829, 0.55825247, 0.45898835, 0.47989123])}
Mean accuracy: 0.25901999024865924
Mean qwkappa: 0.5272843628825239
